In [11]:
from library.load.load import Load
from library.features.features import Polynomial, Standardize, One_Hot_Enc
from library.model.model import Model
from library.preprocessor.preprocessor import PreProcessor
from library.metrics.metrics import Metrics
from library.split.split import Split
import numpy as np

In [12]:
#Loading Data

df = Load('flights_data.csv')

In [13]:
subset = ['flight', 'destination_city', 'source_city', 'class']

# Cleaning Data
df = PreProcessor(df.df)
df.clean(subset) # dropping any NAs in flight, destination city, source city or class
df_clean = df.clean

In [14]:
subset_mean = ['duration','days_left']

# Fill other NAs with mean
df_clean = PreProcessor(df_clean)
df_clean.mean_na(df_clean.df[subset_mean])
df_clean = df_clean.fill

/Users/luisquinonespr/code/CDS_Final/src/library/preprocessor/preprocessor.py:11: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  self.fill = self.df.fillna(self.df.mean())


In [15]:
# Creating features

# We don't want to create features for 'price' so we save it before dropping it
y = np.log(df_clean['price'])
df_feat = df_clean.drop(['Unnamed: 0', 'price'], axis = 1)

# Creating polynomial features
col_names = ['duration', 'days_left']
poly = df_feat[col_names]
poly = Polynomial(poly)
poly = poly.transform(2).drop('1', axis = 1)
col_names2 = ['duration', 'days_left', 'duration^2','duration days_left', 'days_left^2']
df_feat[col_names2] = poly

# We only want to standardize the numerical columns, excluding 'price'
features = df_feat[col_names2]
scaler = Standardize(features)
features = scaler.transform()
df_feat[col_names2] = features

In [16]:
# One-Hot Encoding our categorical variables
df_feat = One_Hot_Enc(df_feat).transform()

In [17]:
# Getting our target variable back to split the data
df_feat['price'] = y

# Splitting between test and train samples
df = Split(df_feat)
test = df.test
train = df.train

In [18]:
# Declaring our X matrices 
X_train = train.drop('price', axis = 1)
X_test = test.drop('price', axis = 1)

# Declaring our target variable
y_train = train['price']
y_test = test['price']

In [19]:
# Training the model
model = Model(X_train, y_train)
model.train_rf()
y_pred_train = model.predict(X_train)
prediction = model.predict(X_test)
test['predictions'] = prediction

In [20]:
# Scoring 
print('Test Score')
score = Metrics.mse(y_test, test['predictions'])

print('Test Score is:' + str(score))

Test Score
Test Score is:6039.7212039906335
